<a href="https://colab.research.google.com/github/Iriansr/covid19/blob/master/covid_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Daily study and modeling of COVID2019 cases


In [0]:
import pandas as pd
import sqlite3 as sql
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import plotly.express as px
import urllib
from bs4 import BeautifulSoup
from urllib.request import urlopen, urlretrieve
from urllib.error import HTTPError
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split

# change dates (updated 17/03 when database changed)
dls = "https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-2020-03-17.xlsx"
urllib.request.urlretrieve(dls, "datos.xlsx")
casos = pd.read_excel('datos.xlsx')

# New Confirmed cases for COVID-19 data visualization.

For representing an interactive graph use the list *custom_countries*.

Valid entries in `*countries_list*`.



In [13]:
## Write here the countries you want to represent
custom_countries = ['Spain', 'Italy','Japan']

casos1 = casos.rename(columns={'Countries and territories': 'Countries'})
countries0 = casos1.Countries.values
countries = np.unique(countries0)
custom = casos1[casos1['Countries'].isin(custom_countries)]
# All countries (coment for changing):
fig = px.line(casos1, x="DateRep", y="Cases", hover_name ="Countries", color ="Countries")
# Custom (coment for changing)
# fig = px.line(custom, x="DateRep", y="Cases", hover_name ="Countries", color ="Countries")
fig.show()

In the next lines, I generate a python dictionary filled by dataframes of every country in the database.
For getting information from each country just:


```
d[key].tail(30)
```
Valid keys are also shown in *countries_list*.


In [0]:
## Generating a suitable dataframe for every country within a dictionary
d = {}
for country in countries:
    d[country] = pd.DataFrame()
for key in d.keys():
  d[key] = casos1.loc[casos1.Countries == key]
  d[key] = d[key].drop(columns=['GeoId'])
  d[key]["DateRep"] = d[key]["DateRep"].astype("datetime64")
  d[key] = d[key][d[key].Cases != 0]
  d[key] = d[key].sort_values(by = 'DateRep')
  d[key]['Cases'] = d[key]['Cases'].cumsum()
  d[key]['Deaths'] = d[key]['Deaths'].cumsum()

# Saving data in SQL databases. 
I will use a folder as a database but you can connect it to your own (see https://www.dataquest.io/blog/python-pandas-databases/) for further treatment.

In [0]:
# Saving the complete database to sql:
connection0 = sql.connect("covid_total.db")

# Generating a table for each country:
dsql = {} 
for country in countries:
  dsql[country] = pd.DataFrame()
for key in dsql.keys():
  dsql[key] = casos1.loc[casos1.Countries == key]
  dsql[key] = dsql[key].drop(columns=['GeoId'])
  dsql[key]["DateRep"] = dsql[key]["DateRep"].astype("datetime64")
  dsql[key] = dsql[key][dsql[key].Cases != 0]
  dsql[key] = dsql[key].sort_values(by = 'DateRep')
  dsql[key]['Cases'] = dsql[key]['Cases'].cumsum()
  dsql[key]['Deaths'] = dsql[key]['Deaths'].cumsum()

connection = sql.connect("covidsql_per_country.db")    # connect to db
crsr = connection.cursor()                             # cursor

for key in dsql.keys():
  dsql[key].to_sql(key,connection, if_exists = 'replace')